# Trabalho 6


## 1. Análise Exploratória e Descritiva (Item b)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import RidgeCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
url = "https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv"
df = pd.read_csv(url)

print(df.head())

   age     sex     bmi  children smoker     region      charges
0   19  female  27.900         0    yes  southwest  16884.92400
1   18    male  33.770         1     no  southeast   1725.55230
2   28    male  33.000         3     no  southeast   4449.46200
3   33    male  22.705         0     no  northwest  21984.47061
4   32    male  28.880         0     no  northwest   3866.85520


In [ ]:
print("Estatísticas Descritivas:")
print(df.describe())

print("\n" + "="*50 + "\n")

# Contando as categorias para as variáveis categóricas
print("Contagem de Categorias (Sexo):")
print(df['sex'].value_counts())
print("\nContagem de Categorias (Fumante):")
print(df['smoker'].value_counts())
print("\nContagem de Categorias (Região):")
print(df['region'].value_counts())

Estatísticas Descritivas:
               age          bmi     children       charges
count  1338.000000  1338.000000  1338.000000   1338.000000
mean     39.207025    30.663397     1.094918  13270.422265
std      14.049960     6.098187     1.205493  12110.011237
min      18.000000    15.960000     0.000000   1121.873900
25%      27.000000    26.296250     0.000000   4740.287150
50%      39.000000    30.400000     1.000000   9382.033000
75%      51.000000    34.693750     2.000000  16639.912515
max      64.000000    53.130000     5.000000  63770.428010


Contagem de Categorias (Sexo):
sex
male      676
female    662
Name: count, dtype: int64

Contagem de Categorias (Fumante):
smoker
no     1064
yes     274
Name: count, dtype: int64

Contagem de Categorias (Região):
region
southeast    364
southwest    325
northwest    325
northeast    324
Name: count, dtype: int64


## 2. Treinamento do Modelo de Regressão Linear Múltipla (Item c)

In [ ]:
df_processados = pd.get_dummies(df, columns=['sex', 'smoker', 'region'], drop_first=True, dtype=int)

# Separação das variáveis preditoras (X) da variável alvo (y)
X = df_processados.drop('charges', axis=1)
y = df_processados['charges']

# Divisão variaveis treino e teste
X_treino, X_teste, y_treino, y_teste = train_test_split(X, y, test_size=0.2, random_state=42)

# Criação e treinamento do modelo
modelo_multiplo = LinearRegression()
modelo_multiplo.fit(X_treino, y_treino)


LinearRegression()

## 4. Processo de Aprimoramento do Modelo (Item d)

In [ ]:
df['log_charges'] = np.log1p(df['charges'])

df_processados = pd.get_dummies(df, columns=['sex', 'smoker', 'region'], drop_first=True, dtype=int)

# Criando os termos de interação usando o df_processados
df_processados['age_smoker'] = df_processados['age'] * df_processados['smoker_yes']
df_processados['bmi_smoker'] = df_processados['bmi'] * df_processados['smoker_yes']

X_improved = df_processados.drop(['charges', 'log_charges'], axis=1)
y_improved = df_processados['log_charges']

X_train, X_test, y_train, y_test = train_test_split(X_improved, y_improved, test_size=0.2, random_state=42)

# Criando e treinando o pipeline com o modelo Ridge
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('ridge', RidgeCV(alphas=np.logspace(-3, 2, 100)))
])

pipeline.fit(X_train, y_train)

log_predictions = pipeline.predict(X_test)
final_predictions = np.expm1(log_predictions)



## 3. Avaliação e Análise Crítica do Modelo (Item e)

In [ ]:
# Calculando as métricas de avaliação sem aprimoramento
previsoes = modelo_multiplo.predict(X_teste)

r2 = r2_score(y_teste, previsoes)
rmse = np.sqrt(mean_squared_error(y_teste, previsoes))
mae = mean_absolute_error(y_teste, previsoes)

print(f"R² (Coeficiente de Determinação): {r2:.4f}")
print(f"RMSE (Raiz do Erro Quadrático Médio): ${rmse:.2f}")
print(f"MAE (Erro Absoluto Médio): ${mae:.2f}")

# Calculando as métricas de avaliação com aprimoramento
r2_aprimorado = r2_score(np.expm1(y_test), final_predictions)
rmse_aprimorado = np.sqrt(mean_squared_error(np.expm1(y_test), final_predictions))
mae_aprimorado = mean_absolute_error(np.expm1(y_test), final_predictions)


print(f"\nR² Aprimorado (Coeficiente de Determinação): {r2_aprimorado:.4f}")
print(f"RMSE Aprimorado (Raiz do Erro Quadrático Médio): ${rmse_aprimorado:.2f}")
print(f"MAE Aprimorado (Erro Absoluto Médio): ${mae_aprimorado:.2f}")

R² (Coeficiente de Determinação): 0.7836
RMSE (Raiz do Erro Quadrático Médio): $5796.28
MAE (Erro Absoluto Médio): $4181.19

R² Aprimorado (Coeficiente de Determinação): 0.8152
RMSE Aprimorado (Raiz do Erro Quadrático Médio): $5355.63
MAE Aprimorado (Erro Absoluto Médio): $2592.68


### Análise dos Resultados Antes

* **R² de 0.7836:** Este resultado é positivo, mostrando que o modelo consegue explicar quase 80% dos motivos que fazem o preço do seguro variar. No entanto, isso também significa que cerca de 20% da variação ainda é um mistério, indicando que outros fatores importantes não foram capturados.

* **Análise do RMSE de \$5.796:** Esse valor alto alerta que o modelo, as vezes, comete erros grandes, provavelmente nos seguros mais caros, mostrando uma certa instabilidade.

* **Análise do MAE de \$4.181:** O MAE é mais direto. Ele nos diz que, na prática, o erro médio da previsão é de cerca de \$4.200, para mais ou para menos. É uma margem de erro bem grande para se trabalhar no dia a dia.

**Conclusão da Análise antes:** O modelo é bom para entender **quais fatores** são importantes, mas **não é preciso o suficiente** para prever o valor exato que uma pessoa vai pagar.


---


###Análise dos Resultados Depois (Versão Aprimorada)
* **R² de 0.8152:** O R² subiu para 0.8152, o que significa que o modelo agora consegue explicar 81,5% das razões que influenciam o preço do seguro. A parte que o modelo não entendia foi reduzida. Isso indica que as melhorias (como a transformação de dados e a inclusão de interações) permitiram ao modelo capturar relações mais complexas entre os dados dos clientes e o custo final.

* **Análise do RMSE de \$5.355,63:** O RMSE caiu de \$5.796 para \$5.355, uma redução de mais de \$440. Isso é um sinal importante de que o modelo se tornou mais estável. Ele não apenas erra menos, mas também comete menos erros "absurdos" ou muito grandes, especialmente naqueles casos de seguros mais caros que antes desestabilizavam as previsões.

* **Análise do MAE de \$2.592,68:** Esta foi a melhoria de maior impacto prático. O erro médio despencou de quase \$4.200 para cerca de \$2.600. Uma redução de quase \$1.600 no erro médio (uma melhora de 38%). Uma margem de erro de \$2.600 ainda existe, o que pode tornar o modelo muito mais gerenciável e aceitável para o dia a dia de uma operação do que a modelo anterior.

Conclusão da Análise Depois: A conclusão mudou. O modelo deixou de ser apenas uma ferramenta para entender quais fatores são importantes e evoluiu para uma ferramenta com posibilidade e capacidade preditiva real. Com um erro médio significativamente menor e maior estabilidade, ele se torna muito mais confiável para estimar o valor que um novo cliente provavelmente pagará, apoiando decisões de negócio de forma mais segura.